In [ ]:
#use "topfind";;
#require "owl-top, jupyter.notebook";;
(* #require "jupyter-archimedes";; *)
(* #require "owl-jupyter";; *)
(* open Owl_jupyter;; *)
open Owl;;

In [15]:
let str_to_elt_fun = function
  | "b" -> fun a -> Dataframe.Bool (bool_of_string a)
  | "i" -> fun a -> Dataframe.Int (int_of_string a)
  | "f" -> fun a -> if a = "" then Dataframe.Float nan else Dataframe.Float (float_of_string a)
  | "s" -> fun a -> Dataframe.String a
  | _   -> failwith "str_to_elt_fun: unsupported type"

let guess_separator lines =
  let sep = [| ','; ' '; '\t'; ';'; ':'; '|' |] in
  (* rank by dividing as many parts as possible *)
  let tmp =
    Array.map
      (fun c ->
        let l = String.split_on_char c lines.(0) in
        c, List.length l)
      sep
  in
  (* sort by decreasing order *)
  Array.sort (fun a b -> snd b - snd a) tmp;
  let sep = Array.map fst tmp in
  let not_sep = ref true in
  let sep_idx = ref 0 in
  while !not_sep = true do
    let c = sep.(!sep_idx) in
    let n = String.split_on_char c lines.(0) |> List.length in
    (try
       Array.iter
         (fun line ->
           let m = String.split_on_char c line |> List.length in
           if m <> n then raise Owl_exception.FOUND)
         lines;
       not_sep := false
     with
    | _exn -> ());
    if !not_sep = true then sep_idx := !sep_idx + 1
  done;
  (* if cannot detect, return comma as default sep *)
  if !not_sep = false then sep.(!sep_idx) else ','


let guess_types sep lines =
  (* Note: no need to add "s" since it is default type *)
  let typ = [| "b"; "i"; "f" |] in
  let num_lines = Array.length lines in
  (* at least two lines because the first one will be dropped *)
  assert (num_lines > 1);
  let num_cols = lines.(0) |> String.trim |> String.split_on_char sep |> List.length in
  (* split into separate columns *)
  let stacks = Array.init num_cols (fun _ -> Owl_utils_stack.make ()) in
  Array.iteri
    (fun i line ->
      if i > 0
      then
        String.trim line
        |> String.split_on_char sep
        |> List.iteri (fun i c -> Owl_utils_stack.push stacks.(i) c))
    lines;
  let cols = Array.map Owl_utils_stack.to_array stacks in
  (* guess the types of columns *)
  Array.mapi
    (fun _i col ->
      let guess_typ = ref "s" in
      (try
         Array.iter
           (fun col_typ ->
             let typ_fun = str_to_elt_fun col_typ in
             let wrong_guess = ref false in
             (try
                Array.iter
                  (fun x ->
                    let y = String.trim x in
                    typ_fun y |> ignore)
                  col
              with
             | _exn -> wrong_guess := true);
             if !wrong_guess = false
             then (
               guess_typ := col_typ;
               raise Owl_exception.FOUND))
           typ
       with
      | _exn -> ());
      !guess_typ)
    cols

let csv_head ?(sep = '\t') head_i lines =
(*     head_i |> ignore;; *)
    let line = lines.(0) in
    line |> String.trim |> String.split_on_char sep |> Array.of_list;;

let of_csv ?sep ?head ?types str =
  let lines = Array.of_list @@ String.split_on_char '\n' str in
  let count = Array.length lines in
  let sep =
    match sep with
    | Some a -> a
    | None   -> guess_separator lines
  in
  let head_i = 0 in
  let head_names =
    match head with
    | Some a -> a
    | None   -> csv_head ~sep head_i lines
  in
  let types =
    match types with
    | Some a -> a
    | None   -> guess_types sep lines
  in
  assert (Array.length head_names = Array.length types);
  let convert_f = Array.map str_to_elt_fun types in
  let dataframe = Dataframe.make head_names in
  let dropped_line = ref 0 in
  for i = head_i + 1 to count - 1 do
    try
        let lines2 = lines.(i) |> Array.make 1 in
        let row = Array.map2 (fun f a -> f a) convert_f lines2 in
        Dataframe.append_row dataframe row
    with
      | _exn ->
        dropped_line := !dropped_line + 1;
        Owl_log.warn "of_csv: fail to parse line#%i" i
  done;
  
  if !dropped_line > 0 then Owl_log.warn "%i lines have been dropped." !dropped_line;
  dataframe


(* let to_csv ?sep x fname =
  let m, n = shape x in
  (* include heads as the first line *)
  let csv = Array.make_matrix (m + 1) n "" in
  csv.(0) <- get_heads x;
  (* dump the data into the table *)
  for i = 0 to m - 1 do
    for j = 0 to n - 1 do
      csv.(i + 1).(j) <- elt_to_str (get x i j)
    done
  done;
  Owl_io.write_csv ?sep csv fname *)

val str_to_elt_fun : string -> string -> Owl.Dataframe.elt = <fun>


val guess_separator : string array -> char = <fun>


val guess_types : char -> string array -> string array = <fun>


val csv_head : ?sep:char -> 'a -> string array -> string array = <fun>


val of_csv :
  ?sep:char ->
  ?head:string array -> ?types:string array -> string -> Owl.Dataframe.t =
  <fun>


In [6]:
String.split_on_char

- : char -> string -> string list = <fun>


No such package: jqdata
